In [0]:
#  Using spark sql, query the parquet file to return the total athletes of each country, then store the data into silver folder
configs = {
  "fs.azure.account.auth.type": "CustomAccessToken",
  "fs.azure.account.custom.token.provider.class": spark.conf.get("spark.databricks.passthrough.adls.gen2.tokenProviderClassName")
}

# Optionally, you can add <directory-name> to the source URI of your mount point.
dbutils.fs.mount(
  source = "abfss://transformtest1@justforgen2.dfs.core.windows.net/",
  mount_point = "/mnt/transformtest1",
  extra_configs = configs)


In [0]:
# create a temporary sql view for querying athletes information
atheletes_data = spark.read.parquet('/mnt/transformtest1/bronze/athletes.parquet')
atheletes_data.createOrReplaceTempView('AtheletesTable')

# Print the total number of athletes  (the number of rows in the athletes data).
# print("Number of athletes: ", atheletes_data.count())
# atheletes_data.show(25, False)

# # Using spark sql, query the parquet file to return the total athletes of each country
num_athletes_by_country=spark.sql("SELECT Country, count(*) AS num FROM AtheletesTable GROUP BY Country ORDER BY num desc")
num_athletes_by_country.show()
out_put="/mnt/silver/"
num_athletes_by_country.write.format("delta").mode("overwrite").save(out_put)


+--------------------+---+
|             Country|num|
+--------------------+---+
|United States of ...|615|
|               Japan|586|
|           Australia|470|
|People's Republic...|401|
|             Germany|400|
|              France|377|
|              Canada|368|
|       Great Britain|366|
|               Italy|356|
|               Spain|324|
|                 ROC|318|
|              Brazil|291|
|         Netherlands|274|
|   Republic of Korea|223|
|         New Zealand|202|
|              Poland|195|
|           Argentina|180|
|        South Africa|171|
|              Mexico|155|
|             Hungary|155|
+--------------------+---+
only showing top 20 rows

